In [1]:
import pandas as pd
from pathlib import Path
from glob import glob
from sklearn.cluster import KMeans

In [2]:
cwd = Path.cwd()
dp = cwd.parent / 'data'
output = cwd.parent / 'output'
files = glob(str(dp / '**/API*.csv'))

In [3]:
# Shows the sum of NaN values in each column for years 2000-2021
# 2019 is the most recent year with the least NaN values
for year in range(2021, 2000, -1):

    year = str(year)
    sf = files[0]

    tdf = pd.read_csv(sf, skiprows=4)
    tdf = tdf.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values=year)

    for file in files[1:]:
        df = pd.read_csv(file, skiprows=4)
        pivot = df.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values=year)
        tdf = tdf.merge(pivot, on=['Country Name', 'Country Code'], how='inner')
    
    print(year, ':',  tdf.isna().sum().sum())
    print(tdf.isna().sum())

2021 : 1598
Indicator Code
AG.LND.TOTL.K2    266
EG.CFT.ACCS.ZS    266
EG.ELC.ACCS.ZS    266
EG.FEC.RNEW.ZS    266
EN.ATM.CO2E.PC    266
SP.DYN.LE00.IN    266
SP.POP.TOTL         2
dtype: int64
2020 : 588
Indicator Code
AG.LND.TOTL.K2      3
EG.CFT.ACCS.ZS     29
EG.ELC.ACCS.ZS      3
EG.FEC.RNEW.ZS    266
EN.ATM.CO2E.PC    266
SP.DYN.LE00.IN     19
SP.POP.TOTL         2
dtype: int64
2019 : 135
Indicator Code
AG.LND.TOTL.K2     3
EG.CFT.ACCS.ZS    29
EG.ELC.ACCS.ZS     2
EG.FEC.RNEW.ZS    53
EN.ATM.CO2E.PC    27
SP.DYN.LE00.IN    19
SP.POP.TOTL        2
dtype: int64
2018 : 136
Indicator Code
AG.LND.TOTL.K2     3
EG.CFT.ACCS.ZS    29
EG.ELC.ACCS.ZS     3
EG.FEC.RNEW.ZS    53
EN.ATM.CO2E.PC    27
SP.DYN.LE00.IN    19
SP.POP.TOTL        2
dtype: int64
2017 : 134
Indicator Code
AG.LND.TOTL.K2     3
EG.CFT.ACCS.ZS    29
EG.ELC.ACCS.ZS     2
EG.FEC.RNEW.ZS    53
EN.ATM.CO2E.PC    27
SP.DYN.LE00.IN    18
SP.POP.TOTL        2
dtype: int64
2016 : 132
Indicator Code
AG.LND.TOTL.K2     3
EG.CFT.A

In [7]:
# Joins all the data into one dataframe for year 2019

sf = files[0]

tdf = pd.read_csv(sf, skiprows=4)
tdf = tdf.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values='2019')

for file in files[1:]:
    df = pd.read_csv(file, skiprows=4)
    pivot = df.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values='2019')
    tdf = tdf.merge(pivot, on=['Country Name', 'Country Code'], how='inner')

In [8]:
tdf = tdf.dropna()
tdf = tdf.reset_index()

In [9]:
tdf

Indicator Code,Country Name,Country Code,AG.LND.TOTL.K2,EG.CFT.ACCS.ZS,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EN.ATM.CO2E.PC,SP.DYN.LE00.IN,SP.POP.TOTL
0,Afghanistan,AFG,652230.0,31.9,97.699997,18.51,0.159824,64.833,38041757.0
1,Albania,ALB,27400.0,80.7,100.000000,40.20,1.692248,78.573,2854191.0
2,Algeria,DZA,2381741.0,99.7,99.500000,0.16,3.977650,76.880,43053054.0
3,Angola,AGO,1246700.0,49.5,45.642799,54.69,0.792137,61.147,31825299.0
4,Antigua and Barbuda,ATG,440.0,100.0,100.000000,0.91,5.354476,77.016,97115.0
...,...,...,...,...,...,...,...,...,...
174,"Venezuela, RB",VEN,882050.0,97.0,99.976814,15.62,3.939566,72.064,28515829.0
175,Vietnam,VNM,313429.0,64.4,99.400002,18.65,3.488313,75.400,96462108.0
176,"Yemen, Rep.",YEM,527970.0,61.3,72.751076,3.11,0.380633,66.125,29161922.0
177,Zambia,ZMB,743390.0,11.2,43.000000,84.50,0.380717,63.886,17861034.0


In [10]:
tdf['POP.DENSE'] = tdf['SP.POP.TOTL'] / tdf['AG.LND.TOTL.K2']

In [11]:
# create kmeans object to categorize the population data
model = KMeans(n_clusters=5, random_state=42).fit(tdf['SP.POP.TOTL'].values.reshape(-1, 1))

# add new column to dataframe with cluster labels
tdf['POP.CAT'] = model.labels_

# Order the countries by pop size
ordered_clusters = tdf['SP.POP.TOTL'].groupby(tdf['POP.CAT']).mean().sort_values()

# List of categories in order of population size
cats = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

c:\Users\chief\.conda\envs\geoprj\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [12]:
ordered_clusters

POP.CAT
0    6.563437e+06
3    4.126422e+07
2    1.095778e+08
4    2.455068e+08
1    1.387081e+09
Name: SP.POP.TOTL, dtype: float64

In [13]:
# Map the categories to the cluster labels
count = 0
for k, v in ordered_clusters.items():
    ordered_clusters[k] = cats[count]
    count += 1

In [14]:
ordered_clusters

POP.CAT
0     Very Low
3          Low
2       Medium
4         High
1    Very High
Name: SP.POP.TOTL, dtype: object

In [16]:
tdf

Indicator Code,Country Name,Country Code,AG.LND.TOTL.K2,EG.CFT.ACCS.ZS,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EN.ATM.CO2E.PC,SP.DYN.LE00.IN,SP.POP.TOTL,POP.DENSE,POP.CAT
0,Afghanistan,AFG,652230.0,31.9,97.699997,18.51,0.159824,64.833,38041757.0,58.325678,3
1,Albania,ALB,27400.0,80.7,100.000000,40.20,1.692248,78.573,2854191.0,104.167555,0
2,Algeria,DZA,2381741.0,99.7,99.500000,0.16,3.977650,76.880,43053054.0,18.076295,3
3,Angola,AGO,1246700.0,49.5,45.642799,54.69,0.792137,61.147,31825299.0,25.527632,3
4,Antigua and Barbuda,ATG,440.0,100.0,100.000000,0.91,5.354476,77.016,97115.0,220.715909,0
...,...,...,...,...,...,...,...,...,...,...,...
174,"Venezuela, RB",VEN,882050.0,97.0,99.976814,15.62,3.939566,72.064,28515829.0,32.329039,3
175,Vietnam,VNM,313429.0,64.4,99.400002,18.65,3.488313,75.400,96462108.0,307.763825,2
176,"Yemen, Rep.",YEM,527970.0,61.3,72.751076,3.11,0.380633,66.125,29161922.0,55.234051,3
177,Zambia,ZMB,743390.0,11.2,43.000000,84.50,0.380717,63.886,17861034.0,24.026465,0


In [17]:
# Replace the cluster labels with the category labels
for k, v in ordered_clusters.items():
    tdf['POP.CAT'] = tdf['POP.CAT'].replace(k, v)

In [21]:
tdf['POP.CAT'].value_counts()

Very Low     124
Low           36
Medium        12
High           5
Very High      2
Name: POP.CAT, dtype: int64

In [22]:
# create kmeans object to categorize the population data
model = KMeans(n_clusters=5, random_state=42).fit(tdf['POP.DENSE'].values.reshape(-1, 1))

# add new column to dataframe with cluster labels
tdf['POP.DENSE.CAT'] = model.labels_

# Order the countries by pop size
ordered_clusters = tdf['POP.DENSE'].groupby(tdf['POP.DENSE.CAT']).mean().sort_values()

# List of categories in order of population size
cats = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

c:\Users\chief\.conda\envs\geoprj\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [23]:
ordered_clusters

POP.DENSE.CAT
0      61.943817
4     255.222486
3     492.786501
2    1673.402168
1    7965.878492
Name: POP.DENSE, dtype: float64

In [24]:
# Map the categories to the cluster labels
count = 0
for k, v in ordered_clusters.items():
    ordered_clusters[k] = cats[count]
    count += 1

In [25]:
ordered_clusters

POP.DENSE.CAT
0     Very Low
4          Low
3       Medium
2         High
1    Very High
Name: POP.DENSE, dtype: object

In [27]:
# Replace the cluster labels with the category labels
for k, v in ordered_clusters.items():
    tdf['POP.DENSE.CAT'] = tdf['POP.DENSE.CAT'].replace(k, v)

In [28]:
tdf

Indicator Code,Country Name,Country Code,AG.LND.TOTL.K2,EG.CFT.ACCS.ZS,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EN.ATM.CO2E.PC,SP.DYN.LE00.IN,SP.POP.TOTL,POP.DENSE,POP.CAT,POP.DENSE.CAT
0,Afghanistan,AFG,652230.0,31.9,97.699997,18.51,0.159824,64.833,38041757.0,58.325678,Low,Very Low
1,Albania,ALB,27400.0,80.7,100.000000,40.20,1.692248,78.573,2854191.0,104.167555,Very Low,Very Low
2,Algeria,DZA,2381741.0,99.7,99.500000,0.16,3.977650,76.880,43053054.0,18.076295,Low,Very Low
3,Angola,AGO,1246700.0,49.5,45.642799,54.69,0.792137,61.147,31825299.0,25.527632,Low,Very Low
4,Antigua and Barbuda,ATG,440.0,100.0,100.000000,0.91,5.354476,77.016,97115.0,220.715909,Very Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...
174,"Venezuela, RB",VEN,882050.0,97.0,99.976814,15.62,3.939566,72.064,28515829.0,32.329039,Low,Very Low
175,Vietnam,VNM,313429.0,64.4,99.400002,18.65,3.488313,75.400,96462108.0,307.763825,Medium,Low
176,"Yemen, Rep.",YEM,527970.0,61.3,72.751076,3.11,0.380633,66.125,29161922.0,55.234051,Low,Very Low
177,Zambia,ZMB,743390.0,11.2,43.000000,84.50,0.380717,63.886,17861034.0,24.026465,Very Low,Very Low


In [30]:
tdf['POP.DENSE.CAT'].value_counts()

Very Low     134
Low           29
Medium        11
High           4
Very High      1
Name: POP.DENSE.CAT, dtype: int64

In [31]:
tdf.to_csv(output / 'joined_data.csv', index=False)